In [ ]:
%env  WORKDIR=/tmp/vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

## Enable LDAP Secret Engine

In [ ]:
! vault secrets enable ldap

## Connect to LDAP with username (bindDN) and password

In [ ]:
%%bash
export LDAP_ADDR=openldap.vault.svc.cluster.local
vault write ldap/config \
    binddn=cn=admin,dc=learn,dc=example \
    bindpass="2LearnVault" \
    url=ldap://$LDAP_ADDR:389

## Create a Role (Static Role)

In [ ]:
%%bash
# Automatic Rotation for static account
vault write ldap/static-role/learn \
    dn='cn=serviceaccount,ou=users,dc=learn,dc=example' \
    username='serviceaccount' \
    rotation_period="1h"

### Request credential via CLI

In [ ]:
! vault read ldap/static-cred/learn

### Request credential via API

In [ ]:
! curl -k --header "X-Vault-Token: $VAULT_TOKEN" --request GET --silent $VAULT_ADDR/v1/ldap/static-cred/learn | jq . -r

> #### Credetials are rotated in a lazy manner, ttl gets reduced, once it comes to 0 with the next request credential are rotated

In [ ]:
! vault read ldap/static-cred/learn

### Manual rotation

In [ ]:
! vault write -f ldap/rotate-role/learn

In [ ]:
! vault read ldap/static-cred/learn

## Dynamic Credentials

In [ ]:
%%bash
cat > ${WORKDIR}/creation.ldif <<EOF
dn: uid={{.Username}},ou=users,dc=learn,dc=example
objectClass: inetOrgPerson
uid: {{.Username}}
cn: {{.Username}}
sn: {{.Username}}
userPassword: {{.Password}}
EOF

In [ ]:
%%bash
cat > ${WORKDIR}/deletion_rollback.ldif <<EOF
dn: uid={{.Username}},ou=users,dc=learn,dc=example
changetype: delete
EOF

In [ ]:
%%bash
# Dynamic Role
vault write ldap/role/dynamic \
     creation_ldif=@${WORKDIR}/creation.ldif \
     deletion_ldif=@${WORKDIR}/deletion_rollback.ldif \
     rollback_ldif=@${WORKDIR}/deletion_rollback.ldif \
     default_ttl=1h

### Request dynamic credentials via CLI

In [ ]:
! vault read ldap/creds/dynamic

In [ ]:
! curl -k --header "X-Vault-Token: $VAULT_TOKEN" --request GET --silent $VAULT_ADDR/v1/ldap/creds/dynamic | jq

### Check leases in Vault

In [ ]:
! vault list sys/leases/lookup/ldap/creds/dynamic

### Revoke individual leases

In [ ]:
! vault list sys/leases/lookup/ldap/creds/dynamic

In [ ]:
%%bash
export LEASE=$(vault list -format=json sys/leases/lookup/ldap/creds/dynamic | jq -r .[0])
vault lease revoke ldap/creds/dynamic/$LEASE

In [ ]:
! vault list sys/leases/lookup/ldap/creds/dynamic

### Revoke all

In [ ]:
! vault lease revoke -prefix ldap/creds/dynamic

# Clean UP